In [ ]:
#upload the files
from google.colab import files
uploaded=files.upload()

Saving Biosynthesis of Amino Acids.md to Biosynthesis of Amino Acids.md
Saving Introduction.md to Introduction.md
Saving Molecules Derived from Amino Acids.md to Molecules Derived from Amino Acids.md
Saving Overview of Nitrogen Metabolism.md to Overview of Nitrogen Metabolism.md


In [ ]:
#create the data directory
import os
import shutil
os.makedirs("Data",exist_ok=True)
for file in os.listdir():
  if file.endswith(".md") or file.endswith(".txt") or file.endswith(".pdf"):
    shutil.move(file,f"Data/{file}")

Data_directory = "Data"
!ls Data

'Biosynthesis of Amino Acids.md'  'Molecules Derived from Amino Acids.md'
 Introduction.md		  'Overview of Nitrogen Metabolism.md'


In [ ]:
print(os.listdir("Data"))

['Overview of Nitrogen Metabolism.md', 'Molecules Derived from Amino Acids.md', 'Introduction.md', 'Biosynthesis of Amino Acids.md']


In [ ]:
!pip install langchain langchain-together streamlit chromadb python-dotenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 74.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 90.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 69.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.4/65.4 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.1/438.1 kB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.0/363.0 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 7.4 MB/s eta 0

In [ ]:
!pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.3 MB/s eta 0:00:00


In [ ]:
!pip install gradio


In [ ]:
# Import dependencies
import os
import gradio as gr
from dotenv import load_dotenv
from langchain.vectorstores import Chroma
from langchain.document_loaders import TextLoader
from langchain_together import ChatTogether, TogetherEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Load environment variables
load_dotenv()
os.environ["TOGETHER_API_KEY"] = "tgp_v1_CUBAgx-EwlF4rpGcqshvzmGbBH_xUdLNSA9jl_BPFmw"
api_key = os.getenv('TOGETHER_API_KEY')

# Set directories
current_directory = os.getcwd()
data_directory = "Data"
persistent_directory = os.path.join(current_directory, "db", "chroma_db_with_metadata")

# Load and split the data
def load_and_split_data():
    bookfiles = [f for f in os.listdir(data_directory) if f.endswith('.md')]
    documents = []

    for bookfile in bookfiles:
        filepath = os.path.join(data_directory, bookfile)
        loader = TextLoader(filepath)
        book_docs = loader.load()
        for doc in book_docs:
            doc.metadata = {'source': bookfile}
            documents.append(doc)

    if not documents:
        raise ValueError("No documents found. Make sure your .md files are not empty!")

    # Split into chunks
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    docs = text_splitter.split_documents(documents)

    # Use TogetherAI embeddings
    embeddings = TogetherEmbeddings(
        model="BAAI/bge-base-en-v1.5",
        together_api_key=api_key
    )

    # Create vector store
    db = Chroma.from_documents(docs, embedding=embeddings, persist_directory=persistent_directory)
    return db.as_retriever(search_type='similarity_score_threshold', search_kwargs={'score_threshold': 0.5, "k": 5})

retriever = load_and_split_data()

# Load the LLM
llm = ChatTogether(
    together_api_key=api_key,
    model="mistralai/Mistral-7B-Instruct-v0.2",
    temperature=0.7
)

# Chat history initialization
chat_history = [{
    "role": "system",
    "content": "You are a helpful AI assistant. Use the context below to answer the user's question. Avoid using em dashes or overly formal language. Sound natural and human, like you're chatting with a student. If the answer is not in the context, say 'I do not know.'"
}]

# Function to generate a response
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage

def convert_to_langchain_messages(chat_history):
    langchain_messages = []
    for message in chat_history:
        role = message["role"]
        content = message["content"]
        if role == "system":
            langchain_messages.append(SystemMessage(content=content))
        elif role == "user":
            langchain_messages.append(HumanMessage(content=content))
        elif role == "assistant":
            langchain_messages.append(AIMessage(content=content))
    return langchain_messages

def generate_chat_response(user_input):
    # Retrieve documents
    docs = retriever.get_relevant_documents(user_input)

    if not docs:
        return "No relevant documents found to answer your question."

    # Combine context
    context = "\n".join([doc.page_content for doc in docs])
    sources = list(set(doc.metadata.get('source') for doc in docs if doc.metadata.get("source")))

    # Add user message to history
    chat_history.append({
        "role": "user",
        "content": f"""Use ONLY the context below to answer the question.
         Context:
         {context}
         Question: {user_input}

         If the answer is not in the context, say "I do not know." """
    })

    # Convert to langchain messages
    langchain_history = convert_to_langchain_messages(chat_history)

    # Get response from LLM
    response = llm(langchain_history)

    # Save assistant message
    chat_history.append({"role": "assistant", "content": response.content})

    # Add sources
    final_response = response.content
    if sources:
        final_response += f"\n\n Sources: {', '.join(sources)}"

    return final_response

iface = gr.Interface(
    fn=generate_chat_response,
    inputs=gr.Textbox(placeholder="Ask a question about your documents...", label="Your Question"),
    outputs="text",
    title=" LEHNINGER RAG Q & A CHATBOT",
    description="Ask questions based on the content of your lehninger documents. The assistant will cite the source documents."
)

iface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://bea50822b778db42ae.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
#testing the rag retriever
input = "what is metabolism"
docs = retriever.get_relevant_documents(input)
print(len(docs))
for doc in docs:
  print(doc.metadata.get("source"))
  print(doc.page_content[:200])

<ipython-input-8-2363443719>:3: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents(input)


3
Overview of Nitrogen Metabolism.md
# Overview of Nitrogen Metabolism

 The biosynthetic pathways leading to amino acids and 
nucleotides share a requirement for nitrogen. Because 
soluble, biologically useful nitrogen compounds are gen
Introduction.md
# Biosynthesis of Amino Acids, Nucleotides, and Related Molecules

Nitrogen ranks behind only carbon, hydrogen, and 
oxygen in its contribution to the mass of living systems. Most of this nitrogen is 
Introduction.md
The pathways described here can be intimidating to 
the beginning biochemistry student. Their complexity 
arises not so much from the chemistry itself, which in 
many cases is well understood, but fro
